# Static Trumpet Black Hole Initial data

**Authors: Terrence Pierre Jacques, Zachariah Etienne, and Ian Ruchlin**

## This module sets up Static Trumpet Black Hole initial data ([Dennison and Baumgarte, 2014 Class. Quantum Grav. 31 117001](https://arxiv.org/abs/1403.5484)). We can convert from spherical to any coordinate system defined in [reference_metric.py](../edit/reference_metric.py) (e.g., SinhSpherical, Cylindrical, Cartesian, etc.) using the [Exact ADM Spherical-or-Cartesian-to-BSSNCurvilinear converter module](Tutorial-ADM_Initial_Data-Converting_Exact_ADM_Spherical_or_Cartesian_to_BSSNCurvilinear.ipynb)

### NRPy+ Source Code for this module: [BSSN/BrillLindquist.py](../edit/BSSN/BrillLindquist.py)

<font color='green'>**All quantities have been validated against the [original SENR code](https://bitbucket.org/zach_etienne/nrpy).**</font>

<a id='top'></a>$$\label{top}$$

### Introduction

These initial data are derived from a family of analytical coordinate systems representing the Schwarzschild spacetime. The coordinates extend smoothly through the black hole event horizon, the spatial coordinates are isotropic (so that the spatial metric can be written as a conformal factor to some power times a flat spatial metric), and, for almost all members of the family, the spatial slices take a so-called $\textit{trumpet geometry}$. Moreover, all expressions are surprisingly simple. This module sets the static trumpet black hole at the origin.


### Table of Contents:  

1. Preliminaries
    1. [Step 1](#step1): Import the core NRPy+ modules that we need, set the dimensionality of the grid with parameter $\text{grid::DIM}$, and declare some of the basic gridfunctions
1. The spatial metric $\gamma_{ij}$
    1. [Step 2.1](#step2p1): Define the conformal factor $\psi$
    1. [Step 2.2](#step2p2): Define and construct nonzero components of $\gamma_{ij}$
1. Extrinsic curvature $K_{ij}$
    1. [Step 3](#step3): Define and construct nonzero components of $K_{ij}$, at the radius $R_0 = M$
1. Lapse function $\alpha$ and shift vector $\beta$
    1. [Step 4](#step4): Construct $\alpha$ and components of $\beta$
1. Code Validation
    1. [Step 5](#step5): NRPy+ Module Code Validation

# Preliminaries

First, we will import the core modules of NRPy that we will need and specify the main gridfunctions we will need. 
<a id='step1'></a>

## Step 1: Set up the needed NRPy+ infrastructure and declare core gridfunctions \[Back to [top](#top)\]

**Input for initial data**:

* The black hole mass $M$.
$$\label{step1}$$

In [1]:
import sympy as sp
import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
from outputC import *
import reference_metric as rfm

# All gridfunctions will be written in terms of spherical coordinates (r, th, ph):
r,th,ph = sp.symbols('r th ph', real=True)

thismodule = "StaticTrumpet"

DIM = 3
par.set_parval_from_str("grid::DIM",DIM)

# Input parameters:
M = par.Cparameters("REAL", thismodule, ["M"])

# Auxiliary variables:
psi0 = sp.symbols('psi0', real=True)

## Step 2.1: Define the conformal factor $\psi$ \[Back to [top](#top)\]
<a id='step2p1'></a>
$$\label{step2p1}$$


The conformal factor, defined in equation 13 of [Dennison and Baumgarte (2014)](https://arxiv.org/abs/1403.5484), setting $R_0 = M$,
$$ \psi = \sqrt{1 + \frac{M}{r}}. $$

In [2]:
# *** The StaticTrumpet conformal factor ***
# Dennison and Baumgarte (2014) Eq. 13
# https://arxiv.org/pdf/1403.5484.pdf

# psi = sqrt{1 + M/r }
psi0 = sp.sqrt(1 + M/r)

<a id='step2p2'></a>

## Step 2.2: Define and construct nonzero components of $\gamma_{ij}$ \[Back to [top](#top)\]
$$\label{step2p2}$$

The spatial metric, defined in equation 15 of [Dennison and Baumgarte (2014)](https://arxiv.org/abs/1403.5484),
$$ \gamma_{ij} = \psi^4 \eta_{ij}, $$

where $\eta_{ij}$ is the flat metric in spherical polar coordinates.

In [3]:
# *** The physical spatial metric in spherical basis ***
# Set the upper-triangle of the matrix...
# Eq. 15
# eta_00 = 1, eta_11 = r^2, eta_22 = r^2 * sin^2 (theta) 
gammaSphDD = ixp.zerorank2()
gammaSphDD[0][0] = psi0**4
gammaSphDD[1][1] = psi0**4 * r**2
gammaSphDD[2][2] = psi0**4 * r**2*sp.sin(th)**2

<a id='step3'></a>

## Step 3: Define and construct nonzero components of $K_{ij}$, at the radius $R_0 = M$ \[Back to [top](#top)\]

$$\label{step3}$$

Components of the extrinsic curvature in spherical basis, defined in equations 19 and 20 of [Dennison and Baumgarte (2014)](https://arxiv.org/abs/1403.5484),

$$ K_{rr} = - \frac{r \left( M-R_0 \right) + MR_0}{r^2 f_1}, $$

<br>

$$ K_{\theta\theta} = \frac{K_{\phi\phi}}{\sin^2 \theta} = f_1, $$

<br>

where $f_1 = \sqrt{2r \left( M-R_0 \right) + R_0 \left( 2M-R_0 \right).}$ Setting $R_0 = M$, these equations reduce to

<br>

$$ K_{rr} = -\frac{M}{r^2}, $$

<br>

$$ K_{\theta\theta} = \frac{K_{\phi\phi}}{\sin^2 \theta} = M. $$


In [4]:
# *** The physical trace-free extrinsic curvature in spherical basis ***
# Set the upper-triangle of the matrix...

# Eq.19 and 20
KSphDD = ixp.zerorank2()

# K_{rr} = - M / r^2
KSphDD[0][0] = -M / r**2

# K_{theta theta} = K_{phi phi} / sin^2 theta = M
KSphDD[1][1] = M

KSphDD[2][2] = M * sp.sin(th)**2

<a id='step4'></a>

## Step 4: Construct $\alpha$ and components of $\beta$ \[Back to [top](#top)\]
$$\label{step4}$$

Laspe function and shift vector components, equation 15 of [Dennison and Baumgarte (2014)](https://arxiv.org/abs/1403.5484), setting $R_0 = M$,
$$ \alpha = \frac{r}{r+M}, $$
<br>
$$ \beta^r = \frac{Mr}{\left (r+M \right)^2}, $$
<br>
$$\beta^\theta = \beta^\phi = 0. $$

In [5]:
# Eq. 15
# alpha = r / (r+M)
alphaSph = r / (r + M)

betaSphU = ixp.zerorank1()
# beta^r = Mr / (r + M)^2
betaSphU[0] = M*r / (r + M)**2

BSphU = ixp.zerorank1()

<a id='step5'></a>

## Step 5: NRPy+ Module Code Validation \[Back to [top](#top)\]
$$\label{step5}$$

Here, as a code validation check, we verify agreement in the SymPy expressions for static trumpet black hole initial data between

1. this tutorial and 
2. the NRPy+ [BSSN.StaticTrumpet](../edit/BSSN/StaticTrumpet.py) module.

In [6]:
# Reset the list of gridfunctions, as registering a gridfunction
#   twice will spawn an error.
gri.glb_gridfcs_list = []

# First we import needed core NRPy+ modules
import reference_metric as rfm
import grid as gri
# Unused, but needed because BrillLindquist() also does tensor rescalings.
rfm.reference_metric()

import BSSN.StaticTrumpet as st
st.StaticTrumpet()

# It is SAFE to ignore the warning(s) from re-initializing parameters.
print("^^^ Ignore the minor warning(s) above. ^^^\n")
print("Consistency check between Brill-Lindquist tutorial and NRPy+ BSSN.BrillLindquist module:")
print("alphaSph - st.alphaSph: Should be zero: "+str(sp.simplify(alphaSph - st.alphaSph)))
for i in range(DIM):
    print("betaSphU["+str(i)+"] - st.betaSphU["+str(i)+"]: Should be zero: "+\
          str(sp.simplify(betaSphU[i] - st.betaSphU[i])))
    print("BSphU["+str(i)+"] - st.BaSphU["+str(i)+"]: Should be zero: "+str(sp.simplify(BSphU[i] - st.BSphU[i])))
    for j in range(DIM):
        print("gammaSphDD["+str(i)+"]["+str(j)+"] - st.gammaSphDD["+str(i)+"]["+str(j)+"]: Should be zero: "+\
              str(sp.simplify(gammaSphDD[i][j] - st.gammaSphDD[i][j])))
        print("KSphDD["+str(i)+"]["+str(j)+"] - st.KSphDD["+str(i)+"]["+str(j)+"]: Should be zero: "+\
              str(sp.simplify(KSphDD[i][j] - st.KSphDD[i][j])))

initialize_param() minor warning: Did nothing; already initialized parameter StaticTrumpet::M
^^^ Ignore the minor warning(s) above. ^^^

Consistency check between Brill-Lindquist tutorial and NRPy+ BSSN.BrillLindquist module:
alphaSph - st.alphaSph: Should be zero: 0
betaSphU[0] - st.betaSphU[0]: Should be zero: 0
BSphU[0] - st.BaSphU[0]: Should be zero: 0
gammaSphDD[0][0] - st.gammaSphDD[0][0]: Should be zero: 0
KSphDD[0][0] - st.KSphDD[0][0]: Should be zero: 0
gammaSphDD[0][1] - st.gammaSphDD[0][1]: Should be zero: 0
KSphDD[0][1] - st.KSphDD[0][1]: Should be zero: 0
gammaSphDD[0][2] - st.gammaSphDD[0][2]: Should be zero: 0
KSphDD[0][2] - st.KSphDD[0][2]: Should be zero: 0
betaSphU[1] - st.betaSphU[1]: Should be zero: 0
BSphU[1] - st.BaSphU[1]: Should be zero: 0
gammaSphDD[1][0] - st.gammaSphDD[1][0]: Should be zero: 0
KSphDD[1][0] - st.KSphDD[1][0]: Should be zero: 0
gammaSphDD[1][1] - st.gammaSphDD[1][1]: Should be zero: 0
KSphDD[1][1] - st.KSphDD[1][1]: Should be zero: 0
gammaSphD

### Output to $\LaTeX$, then PDF

In [7]:
!jupyter nbconvert --to latex --template latex_nrpy_style.tplx Tutorial-ADM_Initial_Data-StaticTrumpet.ipynb
!pdflatex -interaction=batchmode Tutorial-ADM_Initial_Data-StaticTrumpet.tex
!pdflatex -interaction=batchmode Tutorial-ADM_Initial_Data-StaticTrumpet.tex
!pdflatex -interaction=batchmode Tutorial-ADM_Initial_Data-StaticTrumpet.tex
!rm -f Tut*.out Tut*.aux Tut*.log

[NbConvertApp] Converting notebook Tutorial-ADM_Initial_Data-StaticTrumpet.ipynb to latex
[NbConvertApp] Writing 36608 bytes to Tutorial-ADM_Initial_Data-StaticTrumpet.tex
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
